# SVD for Recommendation System 

In [1]:
import pandas as pd
import numpy as np
import time
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split
from joblib import dump, load
from surprise.accuracy import rmse

# 1. Đọc dữ liệu từ file CSV
ratings = pd.read_csv('./ratings.csv', names=['CustomerID', 'StockCode', 'rate'], header=0)

# 2. Chuyển đổi dữ liệu sang định dạng của Surprise
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings[['CustomerID', 'StockCode', 'rate']], reader)

# 3. Chia dữ liệu thành trainset và testset (80% cho huấn luyện, 20% cho kiểm tra)
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# 4. Huấn luyện mô hình SVD
algo = SVD(n_factors=75, reg_all=0.04, random_state=42)
algo.fit(trainset)

# 5. Đánh giá mô hình trên tập huấn luyện
train_predictions = algo.test(trainset.build_testset())  # Dự đoán trên trainset

train_rmse = rmse(train_predictions)

# 6. Đánh giá mô hình trên tập kiểm tra
test_predictions = algo.test(testset)  # Dự đoán trên testset
test_rmse = rmse(test_predictions)


RMSE: 0.7676
RMSE: 0.9125


In [2]:
# Tạo tập dữ liệu đầy đủ 
full_data = data.build_full_trainset()

In [3]:
# 8. Tạo ma trận Yhat (dự đoán rating cho tất cả người dùng và sản phẩm)
n_users = full_data.n_users
n_items = full_data.n_items
Yhat = np.zeros((n_items, n_users))

for user_id in range(n_users):
    for item_id in range(n_items):
        est = algo.predict(uid=full_data.to_raw_uid(user_id), iid=full_data.to_raw_iid(item_id)).est
        Yhat[item_id, user_id] = round(est, 1)

In [4]:
# Lưu ma trận Yhat vào file numpy
np.save('Yhat_SVD.npy', Yhat)

# Để tải lại ma trận Yhat sau này
Yhat_loaded = np.load('Yhat_SVD.npy')
print(Yhat_loaded)
Yhat_loaded.shape[0]


[[4.7 4.2 4.4 ... 4.2 1.7 4.6]
 [4.2 3.9 4.  ... 3.6 1.8 4.3]
 [4.9 4.6 4.6 ... 4.1 2.6 4.7]
 ...
 [4.4 4.1 4.3 ... 3.6 1.9 4.2]
 [4.3 3.9 4.2 ... 3.6 1.7 4.4]
 [4.3 4.  4.  ... 3.7 1.8 4.4]]


3642